## Evaluating the mutation function

In [2]:
## CHOICES FOR M
# Note: k is assumed to be 5 for some of these

# m Description
# 1 Draw all values from the distribution U(s − 0.05 , s + 0.05)
# 2 Draw from the distribution N(s, 0.05)
# 3 Randomly draw all values from the set {−1, −0.5, 0, 0.5, 1}
# 4 Draw all values from the distribution U(−1, 1)
# 5 Draw 2 values from either side of s
# 6 Draw 2 values from either side of s, with 1 distant and 1 near on each side

In [16]:
import BSE
import numpy as np
import csv
import matplotlib.pyplot as plt
import math
# IMPORTANT: used to reload BSE library after making changes to the file BSE.py
# MUST do this between each experiment!
import importlib
importlib.reload(BSE)

# Use this to plot trades of a single experiment
def plot_trades(trial_id):
    prices_filename = trial_id + '_transactions.csv'
    x = np.empty(0)
    y = np.empty(0)
    with open(prices_filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[1])
            price = float(row[2])
            x = np.append(x,time)
            y = np.append(y,price)

    plt.plot(x, y, 'x') 
    
# Use this to run an experiment n times and plot all trades
def n_runs_plot(n, trial_id, start_time, end_time, traders_spec, order_sched, dump_all, verbose):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        BSE.market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, dump_all, verbose)
        
        tdump.close()
        
        if (dump_all):
            with open(trialId + '_transactions.csv', newline='') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    time = float(row[1])
                    price = float(row[2])
                    x = np.append(x,time)
                    y = np.append(y,price)

    plt.plot(x, y, 'x');
    
# Use this to run an experiment n times and plot all trades, from a specified index (useful if failure halfway through experiment)
def n_runs_plot_with_index(n, trial_id, index, start_time, end_time, traders_spec, order_sched, dump_all):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(index, n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        BSE.market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, dump_all, False)
        
        tdump.close()
        
        if (dump_all):
            with open(trialId + '_transactions.csv', newline='') as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    time = float(row[1])
                    price = float(row[2])
                    x = np.append(x,time)
                    y = np.append(y,price)

    plt.plot(x, y, 'x', color='black');
    
def create_trader_spec(buyers_spec, sellers_spec):
    traders_spec = {
        "sellers": sellers_spec,
        "buyers": buyers_spec
    }
    return traders_spec

def create_supply_demand_schedule(start_time, end_time, ranges, stepmode):
    schedule = {
        "from": start_time,
        "to": end_time,
        "ranges": ranges,
        "stepmode": stepmode
    }
    return schedule

def create_order_schedule(supply_schedule,demand_schedule,interval,timemode):
    order_sched = {
        "sup": supply_schedule,
        "dem": demand_schedule,
        "interval": interval,
        "timemode": timemode
    }
    return order_sched

In [17]:
import random

## CREATE OFFSET FUNCTIONS

# Unsure whether offset time t is the total time of experiment, or time into that particular trading day
# Created two offset functions, one for each scenario
# Both should generates volatility at the start of the trading day, or when t is small, but decline as time goes on

def schedule_offsetfn(t):
    sign = random.choice([-1,1])                      
    shock_size = sign * random.random() * 50
    if (t != 0):
        return(int(round((1/t)*shock_size,0)))
    else:
        return(int(round(shock_size,0)))
    
def schedule_offsetfn_with_overall_time(t):
    time_of_day = t % 30                                   # day length is 30
    if (time_of_day <= 5):                                 # shock for first 5 seconds              
        trading_day = (t - time_of_day) / 30               # used to generate deterministic random numbers
        random.seed(trading_day)                           # set seed based on trading day
        sign = random.choice([-1,1])                       # determine whether shock should be +ve or -ve   
        shock_size = sign * random.random() * 200          # set size of shock at start of day
        if (time_of_day != 0):
            shock_size = shock_size *  (1 / (time_of_day)) # converge back to normal price 
        return(int(round(shock_size, 0)))
    else:
        return(0)


In [ ]:
# Define market conditions for experiment, and generate 100 CSV files with the results 
# Note: please change the name of the mutation function in BSE.py that you want to use to mutate_strat
# Make sure to reimport BSE before running this function
def run_for_m(m):

    trial_id = str(m)
    
    k = 5
    eval_time = 600
    total_time = k * eval_time * 10
    
    start_time = 0
    end_time = total_time

    # Set length of trading day
    order_interval = 20 

    buyers_spec = [("ZIP",19),("PRSH",1)]
    sellers_spec = buyers_spec
    traders_spec = create_trader_spec(buyers_spec, sellers_spec)
    
    n_trials = 100
    dump_all = False
    verbose = False
    
    sup_range = [(350, 400,schedule_offsetfn)]

    # Define the supply schedule (how limit prices are distributed to sellers, from a range)
    supply_schedule = [
        create_supply_demand_schedule(start_time, end_time, sup_range, "fixed")
    ]
    demand_schedule = supply_schedule

    # Define the demand schedule (how limit prices are distributed to buyers, from a range)
    order_sched = create_order_schedule(supply_schedule, demand_schedule, order_interval, "drip-poisson")

    n_runs_plot(n_trials, trial_id, start_time, end_time, traders_spec, order_sched, dump_all, verbose)

In [ ]:
run_for_m(1)